In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

# ==============================
# 1. LOAD DATA
# ==============================

data = pd.read_csv("hospital_queue.csv")

# Use only required columns
features = data[['queue_length', 'service_time']]
target = data['waiting_time']

# ==============================
# 2. NORMALIZE DATA
# ==============================

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

# Save scalers
joblib.dump(feature_scaler, "feature_scaler.save")
joblib.dump(target_scaler, "target_scaler.save")

# ==============================
# 3. CREATE SEQUENCES FOR LSTM
# ==============================

TIME_STEPS = 5

def create_sequences(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X, y = create_sequences(features_scaled, target_scaled, TIME_STEPS)

# ==============================
# 4. BUILD LSTM MODEL
# ==============================

model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(TIME_STEPS, 2)))
model.add(Dropout(0.2))

model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

# ==============================
# 5. TRAIN MODEL
# ==============================

history = model.fit(
    X, y,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# ==============================
# 6. SAVE MODEL
# ==============================

model.save("waiting_time_lstm_model.h5")

print("✅ Model training complete and saved!")

# ==============================
# 7. TEST PREDICTION (OPTIONAL)
# ==============================

# Take last sequence
last_sequence = X[-1].reshape(1, TIME_STEPS, 2)

pred_scaled = model.predict(last_sequence)

# Convert back to original value
pred = target_scaler.inverse_transform(pred_scaled)

print("📊 Predicted Waiting Time:", float(pred[0][0]), "minutes")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.1063 - mae: 0.2648 - val_loss: 0.0229 - val_mae: 0.1311
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0235 - mae: 0.1223 - val_loss: 0.0174 - val_mae: 0.1125
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0214 - mae: 0.1165 - val_loss: 0.0142 - val_mae: 0.0985
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0207 - mae: 0.1134 - val_loss: 0.0136 - val_mae: 0.0936
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0203 - mae: 0.1121 - val_loss: 0.0136 - val_mae: 0.0934
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0190 - mae: 0.1093 - val_loss: 0.0139 - val_mae: 0.0932
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0201 - mae: 0.1095 - val_loss: 0.0135 - val_mae: 0.0939
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0195 - mae: 0.1095 - val_loss: 0.0136 - val_mae: 0.0935
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0180 - mae

✅ Model training complete and saved!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
📊 Predicted Waiting Time: 161.4200897216797 minutes
